In [ ]:
import os
import re

import numpy as np

import biobeam
import gputools
import pyopencl
from matplotlib import pyplot as plt
from skimage.io import imsave
from skimage.io import imread_collection

from sim_util import dn_from_signal, random_spots_in_radius, simple_downsample
from run_sim import sim_lightsheet
from calmutils.misc import split_str_digit
import run_sim
from sim_util import random_points_in_volume_min_distance

%matplotlib inline
plt.rcParams['figure.figsize'] = [12,12]

In [ ]:
# set and check which gpu is used
gputools.init_device(id_platform = 0, id_device = 1)
gputools.get_device()

In [ ]:
from sim_util import load_tiff_sequence
img = load_tiff_sequence('/Volumes/davidh-ssd/mv-sim/sim_tissue3/sim-phantom/', downsampling=2)

In [ ]:
img.shape, img.dtype

In [ ]:
# illumination from right side
right_illum = True
# phantom size in microns
physical_dims = (400,400,200)
# LS position
ls_pos = 200
# downsampling
ds_factor = 2

In [ ]:
# load tiff series test data

sequence_pattern = 'sim(\d+).tif'
in_path = '/Volumes/davidh-ssd/mv-sim/sim_tissue/sim-phantom/'

files = os.listdir(in_path)
files.sort(key=lambda f: split_str_digit(f))
files

ic = imread_collection([os.path.join(in_path, f) for f in files])
img = ic.concatenate()

In [ ]:
signal = img
signal_ = simple_downsample(signal, ds_factor)
#signal_ = np.swapaxes(signal_, 0, 1)

signal = np.pad(signal_, ((16,16),(0,0),(0,0)), "constant")

    
dn = dn_from_signal(signal, run_sim.RI_DEFAULT, (0.05, 0.1))

#signal = np.zeros_like(signal)

desc = random_spots_in_radius(100, 3, (np.array(signal.shape)/2).astype(np.int)) + (np.array(signal.shape)/2).astype(np.int)

step = 20
off = 10

a = np.meshgrid(*[range(off, m-32, step) for m in signal.shape])
desc = [np.array(x) + 16 for x in zip(*[b.flat for b in a])]


desc = random_points_in_volume_min_distance([15] * len(signal.shape), [s - 16 for s in signal.shape], 10, 10)
for d in desc:
    #print(d)
    signal[tuple(d)] = 10000
    
from scipy import ndimage as ndi
signal = ndi.gaussian_filter(signal, 0.8)
    
if right_illum:
    signal = np.flip(signal, 2)
    

#create a microscope simulator
m = biobeam.SimLSM_Cylindrical(dn = dn, signal = signal, zfoc_illum=physical_dims[0]/2 - ls_pos,
                       NA_illum= .1, NA_detect=.5,
                       n_volumes=4,
                       #size = physical_dims, n0 = run_sim.RI_DEFAULT)
                       size = [p/ds_factor for p in physical_dims], n0 = run_sim.RI_DEFAULT)

In [ ]:
out = np.zeros_like(signal)[:signal.shape[0] - 16*2]
for plane in range(signal.shape[0] - 16*2):
    cz = plane - (signal.shape[0] - 16*2) // 2
    cz = cz * m._bpm_detect.units[-1]
    image = m.simulate_image_z(cz=cz, psf_grid_dim=(8,8), conv_sub_blocks=(2,2))
    out[plane] = image[16]
    #plt.imshow(image[16])


In [ ]:
if right_illum:
    out = np.flip(out, 2)

In [ ]:
a = dn_from_signal(signal, run_sim.RI_DEFAULT, (0.15, 0.2))
np.max(a) / run_sim.RI_DEFAULT

In [ ]:
a = dn_from_signal(np.pad(signal_, ((16,16),(0,0),(0,0)), "constant"), run_sim.RI_DEFAULT, (0.15/run_sim.RI_DEFAULT, 0.15/run_sim.RI_DEFAULT))
plt.hist(a.ravel(), bins=50)


In [ ]:


from skimage.feature import peak_local_max
from scipy import ndimage as ndi


p = peak_local_max(out / np.max(out),threshold_abs=0.1)

In [ ]:
np.swapaxes(img, 0, 1).shape


In [ ]:
psfs = m.psf_grid_z(cz = -20, grid_dim = (4,4), zslice = 32, with_sheet=False)

In [ ]:
psfs.shape
plt.imshow(np.max(psfs, axis=0))
peak_local_max(psfs, threshold_abs=0.001, min_distance=10)

In [ ]:
p.shape

p
p1 = sorted(list(desc), key=lambda x: tuple(list(x)), reverse=True)

print(p)

p1 - np.array([16, 0, 0])

In [ ]:
def norm_arr(arr):
    return arr / np.max(arr)

f, (ax1, ax2) = plt.subplots(nrows=2)
ax1.imshow(norm_arr(np.sqrt(np.max(out, axis=0))))
ax2.imshow(norm_arr(np.sqrt(np.max(np.flip(signal,2)[16:-16], axis=0))))
out.shape

plt.rcParams['figure.figsize'] = [12,12]

In [ ]:
s = np.sum(out[16:-16], axis=2)
plt.imshow(np.fft.fftshift(np.abs(np.fft.fftn(s)))**0.5 )



In [ ]:
out_fmt = 'sim-bbeam-{}-{}.tif'
out_path = '/Volumes/davidh-ssd/mv-sim/sim_tissue/sim-bbeam-right2/'

if not os.path.exists(out_path):
    os.makedirs(out_path)

out = np.zeros((img.shape[0] - 16*2,) + img.shape[1:], dtype=img.dtype)

for i in range(out.shape[0]):
    
    if i != 155:
        continue
    
    cz = i - (img.shape[0] - 16*2) // 2
    cz = cz * m._bpm_detect.units[-1]

    image = m.simulate_image_z(cz=cz, psf_grid_dim=(16,16), conv_sub_blocks=(8,8))
    out[i] = image[16] if not right_illum else np.flip(image[16], 2)
    
    # save immediately
    imsave(os.path.join(out_path, out_fmt.format("left" if not right_illum else "right", i)), out[i])
    
    print("{} of {} done".format(i+1, out.shape[0]))

In [ ]:
image[16] if not right_illum else np.flip(image[16], 1).shape
#out.shape
np.array(img.shape) - 32

from scipy.ndimage import zoom

zoom(img, [1,2,2]).shape

In [ ]:


simple_downsample(img, [1,2,2]).shape

In [ ]:
# call sim_lightsheet direcctly

if __name__ == '__main__':
    
    base_dir = '/Volumes/davidh-ssd/mv-sim/sim_tissue/'
    
    params = [
        (False, 200, 400),
    ]
    ''' (False, 400, 400),
        (False, 550, 400),
        (True, 250, 400),
        (True, 400, 400),
        (True, 550, 400),
        (False, 250, 600),
        (False, 400, 600),
        (False, 550, 600),
        (True, 250, 600),
        (True, 400, 600),
        (True, 550, 600)
    ]
    '''
    for (right_illum, ls_pos, lam) in params:
        out_dir = 'sim-biobeam2-illum_{}-ls_pos_{}-lam-{}'.format('right' if right_illum else 'left', ls_pos, lam)
        sim_lightsheet(base_dir, out_dir, right_illum=right_illum, ls_pos=ls_pos, lam=lam, planes_to_simulate=[128])
    print('ALL DONE.')
    

In [ ]:
_ = """

pl = pyopencl.get_platforms()[0]
pl.get_devices()
os.environ['PYOPENCL_COMPILER_OUTPUT'] = '1'

gputools.init_device(id_platform = 0, id_device = 1)

signal = read_tiff_stack('/Users/david/Desktop/Image 0-1.tif', False)
signal = np.swapaxes(signal, 0,2)
dn = (signal > 0) * 0.01

#create a microscope simulator
m = biobeam.SimLSM_Cylindrical(dn = dn, signal = signal,
                       NA_illum= .1, NA_detect=.45,
                       n_volumes=1,
                       size = (128,512,512), n0 = 1.33)

image = m.simulate_image_z(cz=0, psf_grid_dim=(16,16), conv_sub_blocks=(2,2))


print(image.shape)
plt.imshow(image[16])
"""